# Here we explore the prediction of various properties like PBE bandgap, HSE bandgap, Energy above hull and if the bandgaps are direct or not in ABX3 type compounds 

## Importing all the necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error


## Converting json file into pandas dataframe

In [3]:
# Replace 'your_file.json' with the path to your JSON file
df = pd.read_json('/Users/suneet/projects/PBE-HSE-prediction/all_ABSe3_calculated_data_2.json')
df.head()

,A,A_HOMO,A_IE,A_IR,A_LUMO,A_OS,A_X,A_Z_radii,A_e_affin,B,...,efermi,density,volume,training_type,HSE_band_gap,PBE_insulator,HSE_insulator,a,b,c
0,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Be,...,2.108769,4.368437,567.890393,set_6,0.0097,0,0,4.075319,9.180483,15.178798
1,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Zn,...,2.301251,5.028065,567.890393,set_6,0.0602,0,0,4.075319,9.180483,15.178798
2,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Hg,...,2.938953,6.609168,567.890393,set_6,0.0175,0,0,4.075319,9.180483,15.178798
3,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Cu,...,2.595679,5.006275,567.890393,set_6,0.0114,0,0,4.075319,9.180483,15.178798
4,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Ni,...,2.617512,4.949518,567.890393,set_6,0.0040,0,0,4.075319,9.180483,15.178798


## Analysing dataset

In [6]:
df.describe()

,A_HOMO,A_IE,A_IR,A_LUMO,A_OS,A_X,A_Z_radii,A_e_affin,B_HOMO,B_IE,...,e_above_hull,efermi,density,volume,HSE_band_gap,PBE_insulator,HSE_insulator,a,b,c
count,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,...,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000
mean,-4.235488,732.696552,1.242293,-1.572004,2.991641,1.783678,2.304447,57.208963,-4.259792,736.419958,...,0.733983,3.440496,5.884782,512.814547,0.293561,0.228840,0.370951,3.890364,9.313622,14.318710
std,1.119252,135.869730,0.212076,1.334446,0.961570,0.432583,0.618047,64.332324,1.101291,131.515365,...,9.971522,4.659803,1.356302,75.065316,0.470728,0.420306,0.483312,0.282350,1.088588,1.914430
min,-6.815000,375.700000,0.810000,-5.355000,1.000000,0.790000,0.795000,-68.000000,-6.815000,375.700000,...,0.000000,0.385257,2.929034,326.254744,0.000000,0.000000,0.000000,3.021577,5.410410,9.238741
25%,-5.194000,650.300000,1.096000,-2.432000,2.000000,1.550000,1.997000,17.000000,-5.194000,652.800000,...,0.136197,2.174115,4.887292,451.634304,0.015200,0.000000,0.000000,3.720540,8.733970,13.448358
50%,-4.368000,736.700000,1.230000,-0.943000,3.000000,1.900000,2.410000,50.000000,-4.417000,736.700000,...,0.251193,3.260095,5.759300,537.933861,0.042900,0.000000,0.000000,3.936074,9.180483,15.178798
75%,-3.528000,833.700000,1.360000,-0.596000,4.000000,2.180000,2.700000,101.000000,-3.584000,833.700000,...,0.520905,4.310645,6.714942,567.890393,0.411000,0.000000,1.000000,4.075319,9.338559,15.178798
max,-2.109000,1011.700000,1.880000,-0.298000,5.000000,2.540000,4.310000,222.800000,-2.109000,1011.700000,...,308.618865,140.937936,10.601874,810.982049,2.863100,1.000000,1.000000,5.534548,13.717645,22.324960


In [7]:
df['PBE_band_gap'].describe()

count    957.000000
mean       0.120515
std        0.251546
min        0.000000
25%        0.006700
50%        0.022600
75%        0.081000
max        1.993400
Name: PBE_band_gap, dtype: float64

### Number of unique datatypes

In [8]:
unique_data_types = df.dtypes.unique().tolist()

print("Unique Data Types in the DataFrame:")
print(unique_data_types)

Unique Data Types in the DataFrame:
[dtype('O'), dtype('float64'), dtype('int64')]


## Data cleaning

### Converting DF with stable, PBE_direct, HSE_direct as number values

In [9]:
df['stable'] = df['stable'].replace({'True': 1, 'False': 0})
print(df['stable'])


0      0
1      0
2      0
3      0
4      0
      ..
952    1
953    0
954    0
955    0
956    0
Name: stable, Length: 957, dtype: int64


In [10]:
df['HSE_direct'] = df['HSE_direct'].replace({'True': 1, 'False': 0})
print(df['HSE_direct'])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
952    NaN
953    NaN
954    NaN
955    NaN
956    NaN
Name: HSE_direct, Length: 957, dtype: float64


In [11]:
df['PBE_direct'] = df['PBE_direct'].replace({'True': 1, 'False': 0})
print(df['PBE_direct'])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
952    NaN
953    NaN
954    NaN
955    NaN
956    NaN
Name: PBE_direct, Length: 957, dtype: float64


### Looking at the unique values in a column

In [12]:
unique_values_stable = df['PBE_direct'].unique()
print("Unique values in the 'PBE_direct' column:", unique_values_stable)


Unique values in the 'PBE_direct' column: [ 0.  1. nan]


### Looking at all the parameters for a single compound we have info about

In [13]:
first_row = df.iloc[0]
print(first_row)

A                           Te
A_HOMO                  -5.952
A_IE                     869.2
A_IR                      1.87
A_LUMO                   -0.56
A_OS                         4
A_X                        2.1
A_Z_radii                 1.67
A_e_affin                190.2
B                           Be
B_HOMO                   -5.61
B_IE                     899.4
B_IR                      0.45
B_LUMO                  -2.017
B_OS                         2
B_X                       1.57
B_Z_radii                 1.08
B_e_affin                -48.0
functional_group       TeBeSe3
mu                     0.22727
tau                    1.12031
PBE_vbm                 2.3778
PBE_cbm                 2.3959
PBE_direct                 0.0
HSE_vbm                 2.2194
HSE_cbm                 2.2291
HSE_direct                 0.0
PBE_band_gap            0.0181
total_energy        -60.424158
e_above_hull          0.773124
stable                       0
efermi                2.108769
density 

### Function to quick search anything

In [14]:
# Specify the element to search for
element_to_search = 'SbRhSe3'

# Filter the DataFrame to get the rows where 'X' is present in column 'B'
result = df[df['functional_group'] == element_to_search]

# Print the value of column 'C' from the result
if not result.empty:
    value_of_column_c = result['HSE_direct'].values[0]
    print(f"The value of column 'C' where 'B' is '{element_to_search}' is: {value_of_column_c}")
else:
    print("no rows were {element_to_search} is present")    

The value of column 'C' where 'B' is 'SbRhSe3' is: 0.0


### Extracting composition using matminer

In [37]:
from matminer.featurizers.conversions import StrToComposition
df = StrToComposition().featurize_dataframe(df, "functional_group")
df.head()

/Users/suneet/Library/Python/3.11/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
StrToComposition: 100%|██████████| 957/957 [00:00<00:00, 9548.06it/s]


,A,A_HOMO,A_IE,A_IR,A_LUMO,A_OS,A_X,A_Z_radii,A_e_affin,B,...,density,volume,training_type,HSE_band_gap,PBE_insulator,HSE_insulator,a,b,c,composition
0,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Be,...,4.368437,567.890393,set_6,0.0097,0,0,4.075319,9.180483,15.178798,"(Te, Be, Se)"
1,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Zn,...,5.028065,567.890393,set_6,0.0602,0,0,4.075319,9.180483,15.178798,"(Te, Zn, Se)"
2,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Hg,...,6.609168,567.890393,set_6,0.0175,0,0,4.075319,9.180483,15.178798,"(Te, Hg, Se)"
3,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Cu,...,5.006275,567.890393,set_6,0.0114,0,0,4.075319,9.180483,15.178798,"(Te, Cu, Se)"
4,Te,-5.952,869.2,1.87,-0.56,4,2.1,1.67,190.2,Ni,...,4.949518,567.890393,set_6,0.0040,0,0,4.075319,9.180483,15.178798,"(Te, Ni, Se)"


### To check the percentage of nan values the columns have

In [38]:
for column in df.columns:
    missing_count = df[column].isnull().sum()

    # Calculate the percentage of missing values
    total_rows = len(df)
    percentage_missing = (missing_count / total_rows) * 100
    print(f"Percentage of missing values in column '{column}': {percentage_missing:.2f}%")


Percentage of missing values in column 'A': 0.00%
Percentage of missing values in column 'A_HOMO': 0.00%
Percentage of missing values in column 'A_IE': 0.00%
Percentage of missing values in column 'A_IR': 0.00%
Percentage of missing values in column 'A_LUMO': 0.00%
Percentage of missing values in column 'A_OS': 0.00%
Percentage of missing values in column 'A_X': 0.00%
Percentage of missing values in column 'A_Z_radii': 0.00%
Percentage of missing values in column 'A_e_affin': 0.00%
Percentage of missing values in column 'B': 0.00%
Percentage of missing values in column 'B_HOMO': 0.00%
Percentage of missing values in column 'B_IE': 0.00%
Percentage of missing values in column 'B_IR': 0.00%
Percentage of missing values in column 'B_LUMO': 0.00%
Percentage of missing values in column 'B_OS': 0.00%
Percentage of missing values in column 'B_X': 0.00%
Percentage of missing values in column 'B_Z_radii': 0.00%
Percentage of missing values in column 'B_e_affin': 0.00%
Percentage of missing valu

### Remove duplicate entries

In [39]:
duplicate_values = df['functional_group'].duplicated()
duplicated_elements = df['functional_group'][duplicate_values]

print("Duplicate elements:")
print(duplicated_elements)
print(duplicated_elements.shape)

Duplicate elements:
705     PTlSe3
707    BiRhSe3
708    BiRuSe3
709    BiCrSe3
710    GeIrSe3
711    GaAuSe3
712    GaCrSe3
713    InIrSe3
714    InCrSe3
715    TlNbSe3
716    LaBiSe3
717    LaInSe3
718    LaTlSe3
719    LaAuSe3
720    LaRuSe3
721    LaFeSe3
722    LaCrSe3
723     LaYSe3
724     YAuSe3
725     YCrSe3
726     YLaSe3
727    CaPtSe3
728    SrPbSe3
729    SrPdSe3
730    SrMnSe3
731    SrReSe3
732    BaSnSe3
733    BaPbSe3
734    BaIrSe3
735    BaReSe3
736    BaMoSe3
737     LiVSe3
942    HgReSe3
953    BiAuSe3
954    TlRhSe3
955    CaPdSe3
956     RbVSe3
Name: functional_group, dtype: object
(37,)


In [40]:
indices_to_remove = df[duplicate_values].index

# Remove rows with duplicate occurrences
df = df.drop(indices_to_remove)

# Print the modified DataFrame
df

,A,A_HOMO,A_IE,A_IR,A_LUMO,A_OS,A_X,A_Z_radii,A_e_affin,B,...,density,volume,training_type,HSE_band_gap,PBE_insulator,HSE_insulator,a,b,c,composition
0,Te,-5.952,869.2,1.870,-0.560,4,2.10,1.670,190.20,Be,...,4.368437,567.890393,set_6,0.0097,0,0,4.075319,9.180483,15.178798,"(Te, Be, Se)"
1,Te,-5.952,869.2,1.870,-0.560,4,2.10,1.670,190.20,Zn,...,5.028065,567.890393,set_6,0.0602,0,0,4.075319,9.180483,15.178798,"(Te, Zn, Se)"
2,Te,-5.952,869.2,1.870,-0.560,4,2.10,1.670,190.20,Hg,...,6.609168,567.890393,set_6,0.0175,0,0,4.075319,9.180483,15.178798,"(Te, Hg, Se)"
3,Te,-5.952,869.2,1.870,-0.560,4,2.10,1.670,190.20,Cu,...,5.006275,567.890393,set_6,0.0114,0,0,4.075319,9.180483,15.178798,"(Te, Cu, Se)"
4,Te,-5.952,869.2,1.870,-0.560,4,2.10,1.670,190.20,Ni,...,4.949518,567.890393,set_6,0.0040,0,0,4.075319,9.180483,15.178798,"(Te, Ni, Se)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Ba,-3.235,502.9,1.610,-2.026,2,0.89,3.402,13.95,Ti,...,5.443971,514.969966,set_1,0.5571,1,1,3.944580,8.885966,14.691850,"(Ba, Ti, Se)"
949,K,-2.330,418.8,1.640,-0.793,1,0.82,3.690,48.40,Ta,...,5.987427,506.891366,set_1,0.9214,1,1,3.923844,8.839255,14.614619,"(K, Ta, Se)"
950,La,-2.885,538.1,1.425,-2.881,3,1.10,3.080,50.00,Rh,...,6.979987,455.522342,set_1,0.4571,1,1,3.786547,8.529965,14.103246,"(La, Rh, Se)"
951,Ba,-3.235,502.9,1.610,-2.026,2,0.89,3.402,13.95,Zr,...,5.450586,567.180352,set_1,0.9872,1,1,4.073620,9.176655,15.172469,"(Ba, Zr, Se)"


## Visualizing PBE bandgap values vs HSE Bandgap values

In [102]:
import plotly.express as px
import plotly.graph_objects as go

# Assuming df is your DataFrame
fig = px.scatter(df, x='PBE_band_gap', y='HSE_band_gap', 
                 title='Scatter Plot of PBE band gap vs HSE band gap',
                 width=1000, height=800,
                 color_discrete_sequence=['royalblue'],
                 labels={'PBE_band_gap': 'PBE Band Gap', 'HSE_band_gap': 'HSE Band Gap'})

# Adding x=y dashed line
fig.add_trace(go.Scatter(
    x=[df['PBE_band_gap'].min(), 2.5],
    y=[df['PBE_band_gap'].min(), 2.5],
    mode='lines',
    line=dict(dash='dot', color='black'),
    showlegend=False
))

fig.layout.template = 'simple_white'

fig.update_layout(
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    legend_title_font_color="green",
    title_font_size = 24,
    title_x=0.5,
    plot_bgcolor='white'

)

fig.update_xaxes(title_font_family="Arial",tickfont=dict(color='black'), tickfont_size = 20, title_font_size = 18)
fig.update_yaxes(title_font_family="Arial",tickfont=dict(color='black'), tickfont_size = 20, title_font_size = 18)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)


## Making stability values with own parameters


In [91]:
df['stable2'] = (df['e_above_hull'] <= 0.05).astype(int)

In [92]:
count_of_ones = df['stable2'].sum()
print("Number of 1's in column 'stable':", count_of_ones)

Number of 1's in column 'stable': 59


## Preparing X,y 

### For prediciting PBE bandgap energy


In [103]:
excluded_incHSE = ["A", "B", "functional_group", "training_type", "PBE_band_gap", "composition", "PBE_direct", "stable2", "HSE_direct"] #, "total_energy"] #, "stable"] #, "volume", "c"]
excluded_excHSE = ["A", "B", "functional_group", "training_type", "PBE_band_gap", "composition", "HSE_direct", "PBE_direct", "stable2", "HSE_band_gap", "HSE_cbm", "HSE_vbm"] #, "total_energy"] #, "stable"] #, "volume", "c"]
y_PBE = df['PBE_band_gap'].values
X_PBE = df.drop(excluded_incHSE, axis=1)
X2_PBE = df.drop(excluded_excHSE, axis=1)
print("There are {} possible descriptors:\n\n{}".format(X_PBE.shape[1], X_PBE.columns.values))
print("There are {} possible descriptors:\n\n{}".format(X2_PBE.shape[1], X2_PBE.columns.values))

There are 34 possible descriptors:

['A_HOMO' 'A_IE' 'A_IR' 'A_LUMO' 'A_OS' 'A_X' 'A_Z_radii' 'A_e_affin'
 'B_HOMO' 'B_IE' 'B_IR' 'B_LUMO' 'B_OS' 'B_X' 'B_Z_radii' 'B_e_affin' 'mu'
 'tau' 'PBE_vbm' 'PBE_cbm' 'HSE_vbm' 'HSE_cbm' 'total_energy'
 'e_above_hull' 'stable' 'efermi' 'density' 'volume' 'HSE_band_gap'
 'PBE_insulator' 'HSE_insulator' 'a' 'b' 'c']
There are 31 possible descriptors:

['A_HOMO' 'A_IE' 'A_IR' 'A_LUMO' 'A_OS' 'A_X' 'A_Z_radii' 'A_e_affin'
 'B_HOMO' 'B_IE' 'B_IR' 'B_LUMO' 'B_OS' 'B_X' 'B_Z_radii' 'B_e_affin' 'mu'
 'tau' 'PBE_vbm' 'PBE_cbm' 'total_energy' 'e_above_hull' 'stable' 'efermi'
 'density' 'volume' 'PBE_insulator' 'HSE_insulator' 'a' 'b' 'c']


### For prediciting HSE bandgap energy


In [45]:
excluded_incHSEIN = ["A", "B", "functional_group", "training_type", "HSE_band_gap", "composition", "HSE_direct", "PBE_direct", "stable2", 'HSE_vbm', 'HSE_cbm'] #, "total_energy"] #, "stable"] #, "volume", "c"]
excluded_excHSEIN = ["A", "B", "functional_group", "training_type", "HSE_band_gap", "composition", "HSE_direct", "PBE_direct", "stable2", 'HSE_vbm', 'HSE_cbm', 'HSE_insulator'] #, "total_energy"] #, "stable"] #, "volume", "c"]
y_HSE = df['HSE_band_gap'].values
X_HSE = df.drop(excluded_incHSEIN, axis=1)
X2_HSE = df.drop(excluded_excHSEIN, axis=1)
print("There are {} possible descriptors:\n\n{}".format(X_HSE.shape[1], X_HSE.columns.values))
print("There are {} possible descriptors:\n\n{}".format(X2_HSE.shape[1], X2_HSE.columns.values))

There are 32 possible descriptors:

['A_HOMO' 'A_IE' 'A_IR' 'A_LUMO' 'A_OS' 'A_X' 'A_Z_radii' 'A_e_affin'
 'B_HOMO' 'B_IE' 'B_IR' 'B_LUMO' 'B_OS' 'B_X' 'B_Z_radii' 'B_e_affin' 'mu'
 'tau' 'PBE_vbm' 'PBE_cbm' 'PBE_band_gap' 'total_energy' 'e_above_hull'
 'stable' 'efermi' 'density' 'volume' 'PBE_insulator' 'HSE_insulator' 'a'
 'b' 'c']
There are 31 possible descriptors:

['A_HOMO' 'A_IE' 'A_IR' 'A_LUMO' 'A_OS' 'A_X' 'A_Z_radii' 'A_e_affin'
 'B_HOMO' 'B_IE' 'B_IR' 'B_LUMO' 'B_OS' 'B_X' 'B_Z_radii' 'B_e_affin' 'mu'
 'tau' 'PBE_vbm' 'PBE_cbm' 'PBE_band_gap' 'total_energy' 'e_above_hull'
 'stable' 'efermi' 'density' 'volume' 'PBE_insulator' 'a' 'b' 'c']


## Choosing model based on 10-fold crossvalidation scores

In [46]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

# Assuming X_PBE and y_PBE are your features and target variables

# XGBoost Regressor
xgb_reg = XGBRegressor(n_estimators=200, random_state=22, learning_rate=0.5)

# Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=200, random_state=21)

# Linear Regression
lr_reg = LinearRegression()

# Models
models = [xgb_reg, rf_reg, lr_reg]

for model in models:
    # Use 10-fold cross-validation (90% training, 10% test)
    crossvalidation = KFold(n_splits=10, shuffle=True, random_state=8)
    scores_mse = cross_val_score(model, X_PBE, y_PBE, scoring='neg_mean_squared_error', cv=crossvalidation, n_jobs=1)
    scores_r2 = cross_val_score(model, X_PBE, y_PBE, scoring='r2', cv=crossvalidation, n_jobs=1)

    rmse_scores = [np.sqrt(abs(s)) for s in scores_mse]

    print(f'Model: {type(model).__name__}')
    print('Cross-validation results:')
    print(f'Folds: {len(scores_mse)}, mean R2: {np.mean(np.abs(scores_r2)):.3f}')
    print(f'Folds: {len(scores_mse)}, mean RMSE: {np.mean(np.abs(rmse_scores)):.3f}\n')


Model: XGBRegressor
Cross-validation results:
Folds: 10, mean R2: 0.916
Folds: 10, mean RMSE: 0.069

Model: RandomForestRegressor
Cross-validation results:
Folds: 10, mean R2: 0.910
Folds: 10, mean RMSE: 0.071

Model: LinearRegression
Cross-validation results:
Folds: 10, mean R2: 5.811
Folds: 10, mean RMSE: 0.193



## XGBoost

## Only to check optimum values for the XGBoost model


In [36]:
'''from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

best_rmse = float('inf')
best_seed = None

for seed in range(1, 43):
    # Split the data into train and test sets using the current seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.09, random_state=seed)

    # Extract and drop the 'functional_group' column
    train_formula = X_train[('functional_group', 'stable')]
    X_train = X_train.drop(['functional_group', 'stable'], axis=1)
    test_formula = X_test['functional_group']
    X_test = X_test.drop(['functional_group', 'stable'], axis=1)

    # Create an XGBoost regressor
    xgb_reg = XGBRegressor(n_estimators=100, random_state=5)
    xgb_reg.fit(X_train, y_train)

    # Calculate RMSE on the test set
    current_rmse = np.sqrt(mean_squared_error(y_true=y_test, y_pred=xgb_reg.predict(X_test)))

    # Update best RMSE and seed if the current RMSE is better
    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_seed = seed

print(f'Best RMSE: {best_rmse:.3f} obtained with random seed: {best_seed}')'''


Best RMSE: 0.031 obtained with random seed: 6


## For prediciting PBE bandgap

### Predicting PBE Bandgap including HSE features

In [104]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming X, y, and df are already defined

# Add 'functional_group' to X
X_PBE['functional_group'] = df['functional_group']

# Add 'stable' to X
X_PBE['stable'] = df['stable']

# Specify the test size
test_size = 0.2

# Split the data into train and test sets while maintaining the proportion of 'stable'
X_train, X_test, y_train, y_test = train_test_split(X_PBE, y_PBE, test_size=test_size, random_state=21, stratify=X_PBE['stable'])

# Extract and drop the 'functional_group' and 'stable' columns
train_formula = X_train[['functional_group', 'stable']]
X_train = X_train.drop(['functional_group', 'stable'], axis=1)
test_formula = X_test[['functional_group', 'stable']]
X_test = X_test.drop(['functional_group', 'stable'], axis=1)

# Create an XGBoost regressor
xgb_reg = XGBRegressor(n_estimators=200, random_state=9, learning_rate=0.5)
xgb_reg.fit(X_train, y_train)

# Get fit statistics
print('Training R2 = ' + str(round(xgb_reg.score(X_train, y_train), 3)))
print('Training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_train, y_pred=xgb_reg.predict(X_train))))
print('Test R2 = ' + str(round(xgb_reg.score(X_test, y_test), 3)))
print('Test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=xgb_reg.predict(X_test))))


Training R2 = 1.0
Training RMSE = 0.001
Test R2 = 0.94
Test RMSE = 0.057


### Graph for predicting PBE Bandgap including HSE features

In [114]:
import plotly.graph_objects as go

# Predict on training and test sets
y_train_pred = xgb_reg.predict(X_train)
y_test_pred = xgb_reg.predict(X_test)


# Create a parity plot for test set
test_set_df_ = pd.DataFrame({'y_test': y_test, 'y_test_pred': y_test_pred})

# Merge the DataFrame with the main DataFrame 'df' to get 'hse_direct' values
test_set_df_ = pd.merge(test_set_df_, df[['PBE_direct']], left_index=True, right_index=True, how='left')

parity_test_fig = go.Figure()

# Create a color list based on pbe_direct values
colors = []

for index, row in test_set_df_.iterrows():
    pbe_direct_val = row['PBE_direct']
    if pd.isna(pbe_direct_val):
        colors.append('red')
    elif pbe_direct_val == 1:
        colors.append('green')
    else:
        colors.append('black')

parity_test_fig.add_trace(go.Scatter(
    x=test_set_df_['y_test'],
    y=test_set_df_['y_test_pred'],
    mode='markers',
    name='Test Set',
    marker=dict(size=6, color=colors)
))

parity_test_fig.add_trace(go.Scatter(
    x=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    y=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    mode='lines',
    name='Ideal Prediction',
    line=dict(dash='dash', color='red', width=1.5)
))

# Add custom legend entry
parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='PBE direct'
))

parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='black'),
    name='PBE indirect'
))

parity_test_fig.update_layout(
    template='simple_white',  # set template to white background
    title='Parity Plot for predicting PBE Bandgap including HSE features',
    xaxis_title='Actual Values',
    yaxis_title='Predicted Values',
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    title_font_size = 24,
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    legend_title_font_color="green",
    xaxis_title_font_family = "arial",
    yaxis_title_font_family = "arial",
    title_x = 0.5,
    showlegend=False,  # hide legend
    width=1000,  # Set the width of the plot
    height=800,
)

#fig.update_xaxes(title_font_family="Arial",tickfont=dict(color='black'), tickfont_size = 16, title_font_size = 20)
#fig.update_yaxes(title_font_family="Arial",tickfont=dict(color='black'), tickfont_size = 16, title_font_size = 20)

textt = "Black points represent PBE bandgap is indirect<br>Green points represent PBE bandgap is direct"

parity_test_fig.add_annotation(
    text=textt,  # Replace with your desired text
    x= 1,  # X-coordinate position (0 to 1, where 0 is left and 1 is right)
    y= 0.05,  # Y-coordinate position (1 is top of the plot, increase to move downward)
    xref="paper",  # Use "paper" for relative x-coordinates
    yref="paper",  # Use "paper" for relative y-coordinates
    showarrow=False,  # Do not show an arrow
    font=dict(family="Arial", size=12, color="black"),  # Customize font properties
    bgcolor="rgba(255, 255, 255, 0.7)",  # Set background color
    bordercolor="black",  # Set border color
    borderwidth=1,  # Set border width
)

#fig.update_xaxes(title_font_family="Arial",tickfont=dict(color='black'))
#fig.update_yaxes(title_font_family="Arial",tickfont=dict(color='black'))

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

parity_test_fig.show(config=config)





### Plotting Importances by features of the PBE prediction model including HSE features

In [124]:
import plotly.graph_objects as go

feature_importances = xgb_reg.feature_importances_
included = X_PBE.columns.values
indices = np.argsort(feature_importances)[::-1]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=included[indices][:10],
    y=feature_importances[indices][:10],
    marker=dict(color='royalblue'),
    #text=feature_importances[indices][:10],
    #textposition='auto',
))

fig.update_layout(
    xaxis=dict(
        title='Features',
        tickfont=dict(size=14),
        tickangle=45  
    ),
    yaxis_title='Importance (%)',
    title='Feature Importances for Predicting PBE Features including HSE parameters',
    font=dict(family="Arial", size=14, color="black"),
    title_x = 0.5,
    width=1000,
    height=800,
    showlegend=False,
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)


### Predicting PBE Bandgap excluding HSE features

In [125]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming X, y, and df are already defined

# Add 'functional_group' to X
X2_PBE['functional_group'] = df['functional_group']

# Add 'stable' to X
X2_PBE['stable'] = df['stable']

# Specify the test size
test_size = 0.2

# Split the data into train and test sets while maintaining the proportion of 'stable'
X_train, X_test, y_train, y_test = train_test_split(X2_PBE, y_PBE, test_size=test_size, random_state=20, stratify=X2_PBE['stable'])

# Extract and drop the 'functional_group' and 'stable' columns
train_formula = X_train[['functional_group', 'stable']]
X_train = X_train.drop(['functional_group', 'stable'], axis=1)
test_formula = X_test[['functional_group', 'stable']]
X_test = X_test.drop(['functional_group', 'stable'], axis=1)

# Create an XGBoost regressor
xgb_reg = XGBRegressor(n_estimators=200, random_state=9, learning_rate=0.1, max_depth= 3)
xgb_reg.fit(X_train, y_train)

# Get fit statistics
print('Training R2 = ' + str(round(xgb_reg.score(X_train, y_train), 3)))
print('Training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_train, y_pred=xgb_reg.predict(X_train))))
print('Test R2 = ' + str(round(xgb_reg.score(X_test, y_test), 3)))
print('Test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=xgb_reg.predict(X_test))))


Training R2 = 0.987
Training RMSE = 0.027
Test R2 = 0.901
Test RMSE = 0.094


### Graph for predicting PBE Bandgap excluding HSE features

In [132]:
import plotly.graph_objects as go

# Predict on training and test sets
y_train_pred = xgb_reg.predict(X_train)
y_test_pred = xgb_reg.predict(X_test)


# Create a parity plot for test set
test_set_df_ = pd.DataFrame({'y_test': y_test, 'y_test_pred': y_test_pred})

# Merge the DataFrame with the main DataFrame 'df' to get 'hse_direct' values
test_set_df_ = pd.merge(test_set_df_, df[['PBE_direct']], left_index=True, right_index=True, how='left')

parity_test_fig = go.Figure()

# Create a color list based on pbe_direct values
colors = []

for index, row in test_set_df_.iterrows():
    pbe_direct_val = row['PBE_direct']
    if pd.isna(pbe_direct_val):
        colors.append('red')
    elif pbe_direct_val == 1:
        colors.append('green')
    else:
        colors.append('black')

parity_test_fig.add_trace(go.Scatter(
    x=test_set_df_['y_test'],
    y=test_set_df_['y_test_pred'],
    mode='markers',
    name='Test Set',
    marker=dict(size=6, color=colors)
))

parity_test_fig.add_trace(go.Scatter(
    x=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    y=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    mode='lines',
    name='Ideal Prediction',
    line=dict(dash='dash', color='red', width=1.5)
))

# Add custom legend entry
parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='PBE direct'
))

parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='black'),
    name='PBE indirect'
))

parity_test_fig.update_layout(
    template='simple_white',  
    title='Parity Plot for predicting PBE Bandgap excluding HSE features',
    xaxis_title='Actual Values',
    yaxis_title='Predicted Values',
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_size = 20,
    title_font_color="black",
    title_x = 0.5,
    xaxis_title_font_size = 18,
    yaxis_title_font_size = 18,
    legend_title_font_color="green",
    xaxis_title_font_family = "arial",
    yaxis_title_font_family = "arial",
    showlegend=False,  
    width=1000,  
    height=800,
)

textt = "Black points represent PBE bandgap is indirect<br>Green points represent PBE bandgap is direct"

parity_test_fig.add_annotation(
    text=textt,  
    x= 1,  
    y= 0.05,  
    xref="paper",  
    yref="paper",  
    showarrow=False, 
    font=dict(family="Arial", size=12, color="black"),  
    bgcolor="rgba(255, 255, 255, 0.7)",  
    bordercolor="black",  
    borderwidth=1,  
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

parity_test_fig.show(config=config)





### Plotting Importances by features of the PBE prediction model excluding HSE features

In [135]:
import plotly.graph_objects as go

feature_importances = xgb_reg.feature_importances_
included = X2_PBE.columns.values
indices = np.argsort(feature_importances)[::-1]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=included[indices][:10],
    y=feature_importances[indices][:10],
    marker=dict(color='royalblue'),
    #text=feature_importances[indices][:10],
    #textposition='auto',
))

fig.update_layout(
    xaxis=dict(
        title='Features',
        tickfont=dict(size=13),
        tickangle=45  
    ),
    yaxis_title='Importance (%)',
    title='Feature Importances for Predicting PBE Features excluding HSE parameters',
    font=dict(family="Arial", size=14, color="black"),
    title_x = 0.5,
    width=1000,
    height=800,
    showlegend=False,
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)


## For prediciting HSE bandgap

### Predicting HSE bandgap including HSE insulator

In [136]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming X, y, and df are already defined

X_HSE['functional_group'] = df['functional_group']

# Add 'stable' to X
X_HSE['stable'] = df['stable']

# Specify the test size
test_size = 0.1

# Split the data into train and test sets
X__train, X__test, y__train, y__test = train_test_split(X_HSE, y_HSE, test_size=test_size, random_state=2, stratify= X_HSE['stable'])

# Extract and drop the 'functional_group' column
train_formula = X__train['functional_group']
X__train = X__train.drop('functional_group', axis=1)
test__formula = X__test['functional_group']
X__test = X__test.drop('functional_group', axis=1)

# Create an XGBoost regressor
xgb__reg = XGBRegressor(n_estimators=250, random_state=6, learning_rate = 0.1)
xgb__reg.fit(X__train, y__train)

# Get fit statistics
print('Training R2 = ' + str(round(xgb__reg.score(X__train, y__train), 3)))
print('Training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y__train, y_pred=xgb__reg.predict(X__train))))
print('Test R2 = ' + str(round(xgb__reg.score(X__test, y__test), 3)))
print('Test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y__test, y_pred=xgb__reg.predict(X__test))))



Training R2 = 1.0
Training RMSE = 0.004
Test R2 = 0.945
Test RMSE = 0.104


### Parity plot for HSE Bandgap including HSE insulator

In [141]:
import plotly.graph_objects as go

# Predict on training and test sets
y__train_pred = xgb__reg.predict(X__train)
y__test_pred = xgb__reg.predict(X__test)


# Create a parity plot for test set
test_set_df = pd.DataFrame({'y_test': y__test, 'y_test_pred': y__test_pred})

# Merge the DataFrame with the main DataFrame 'df' to get 'hse_direct' values
test_set_df = pd.merge(test_set_df, df[['HSE_direct']], left_index=True, right_index=True, how='left')

parity_test_fig = go.Figure()

# Create a color list based on pbe_direct values
colors = []

for index, row in test_set_df.iterrows():
    hse_direct_val = row['HSE_direct']
    if pd.isna(hse_direct_val):
        colors.append('red')
    elif hse_direct_val == 1:
        colors.append('green')
    else:
        colors.append('black')

parity_test_fig.add_trace(go.Scatter(
    x=test_set_df['y_test'],
    y=test_set_df['y_test_pred'],
    mode='markers',
    name='Test Set',
    marker=dict(size=6, color=colors)
))

parity_test_fig.add_trace(go.Scatter(
    x=[min(test_set_df['y_test']), max(test_set_df['y_test'])],
    y=[min(test_set_df['y_test']), max(test_set_df['y_test'])],
    mode='lines',
    name='Ideal Prediction',
    line=dict(dash='dash', color='red', width=1.5)
))

# Add custom legend entry
parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='HSE direct'
))

parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='black'),
    name='HSE indirect'
))

parity_test_fig.update_layout(
    template='simple_white',  
    title='Parity plot for HSE Bandgap including HSE insulator',
    xaxis_title='Actual Values',
    yaxis_title='Predicted Values',
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    title_font_size = 20,
    xaxis_title_font_size = 18,
    yaxis_title_font_size = 18,
    title_x = 0.5,
    legend_title_font_color="green",
    xaxis_title_font_family = "arial",
    yaxis_title_font_family = "arial",
    showlegend=False,  
    width=1000,  
    height=800,
)

textt = "Black points represent HSE bandgap is indirect<br>Green points represent HSE bandgap is direct"

parity_test_fig.add_annotation(
    text=textt,  
    x= 1,  
    y= 0.05,  
    xref="paper",  
    yref="paper",  
    showarrow=False, 
    font=dict(family="Arial", size=12, color="black"),  
    bgcolor="rgba(255, 255, 255, 0.7)",  
    bordercolor="black",  
    borderwidth=1,  
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

parity_test_fig.show(config=config)

### Feature importances for HSE bandgap prediction including HSE insulator

In [150]:
import plotly.graph_objects as go

feature_importances = xgb__reg.feature_importances_
included = X_HSE.columns.values
indices = np.argsort(feature_importances)[::-1]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=included[indices][:10],
    y=feature_importances[indices][:10],
    marker=dict(color='royalblue'),
    #text=feature_importances[indices][:10],
    #textposition='auto',
))

fig.update_layout(
    xaxis=dict(
        title='Features',
        tickfont=dict(size=14),
        tickangle = 45,  # Adjust the font size for tick labels as needed
    ),
    yaxis_title='Importance (%)',
    title='Feature Importances for Predicting HSE Features including HSE insulator',
    font=dict(family="Arial", size=14, color="black"),
    title_x = 0.5,
    width=1000,
    height=800,
    showlegend=False,
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)


### Predicting HSE bandgap excluding HSE insulator

In [151]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming X, y, and df are already defined

X2_HSE['functional_group'] = df['functional_group']

# Add 'stable' to X
X2_HSE['stable'] = df['stable']

# Specify the test size
test_size = 0.1

# Split the data into train and test sets
X__train, X__test, y__train, y__test = train_test_split(X2_HSE, y_HSE, test_size=test_size, random_state=17, stratify= X2_HSE['stable'])

# Extract and drop the 'functional_group' column
train_formula = X__train['functional_group']
X__train = X__train.drop('functional_group', axis=1)
test__formula = X__test['functional_group']
X__test = X__test.drop('functional_group', axis=1)

# Create an XGBoost regressor
xgb__reg = XGBRegressor(n_estimators=250, random_state=6, learning_rate = 0.1)
xgb__reg.fit(X__train, y__train)

# Get fit statistics
print('Training R2 = ' + str(round(xgb__reg.score(X__train, y__train), 3)))
print('Training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y__train, y_pred=xgb__reg.predict(X__train))))
print('Test R2 = ' + str(round(xgb__reg.score(X__test, y__test), 3)))
print('Test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y__test, y_pred=xgb__reg.predict(X__test))))



Training R2 = 1.0
Training RMSE = 0.008
Test R2 = 0.893
Test RMSE = 0.177


### Optimizing parameters for prediction

In [71]:
'''from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming X, y, and df are already defined

X2_HSE['functional_group'] = df['functional_group']

# Add 'stable' to X
X2_HSE['stable'] = df['stable']

# Specify the test size
test_size = 0.1

best_r2_score = -float('inf')
best_seed = None

for seed in range(44):
    # Split the data into train and test sets with different random seed
    X__train, X__test, y__train, y__test = train_test_split(X2_HSE, y_HSE, test_size=test_size, random_state=seed, stratify=X2_HSE['stable'])

    # Extract and drop the 'functional_group' column
    train_formula = X__train['functional_group']
    X__train = X__train.drop('functional_group', axis=1)
    test__formula = X__test['functional_group']
    X__test = X__test.drop('functional_group', axis=1)

    # Create an XGBoost regressor
    xgb__reg = XGBRegressor(n_estimators=250, random_state=6, learning_rate=0.1)
    xgb__reg.fit(X__train, y__train)

    # Get test R2 score
    test_r2 = xgb__reg.score(X__test, y__test)
    
    # Update best score and seed if current score is better
    if test_r2 > best_r2_score:
        best_r2_score = test_r2
        best_seed = seed

# Print the best result
print('Best Random Seed:', best_seed)
print('Best Test R2 Score:', best_r2_score)'''


Best Random Seed: 17
Best Test R2 Score: 0.8927719442095472


### Parity plot for HSE Bandgap excluding HSE insulator

In [160]:
import plotly.graph_objects as go

# Predict on training and test sets
y__train_pred = xgb__reg.predict(X__train)
y__test_pred = xgb__reg.predict(X__test)


# Create a parity plot for test set
test_set_df = pd.DataFrame({'y_test': y__test, 'y_test_pred': y__test_pred})

# Merge the DataFrame with the main DataFrame 'df' to get 'hse_direct' values
test_set_df = pd.merge(test_set_df, df[['HSE_direct']], left_index=True, right_index=True, how='left')

parity_test_fig = go.Figure()

# Create a color list based on pbe_direct values
colors = []

for index, row in test_set_df.iterrows():
    hse_direct_val = row['HSE_direct']
    if pd.isna(pbe_direct_val):
        colors.append('red')
    elif hse_direct_val == 1:
        colors.append('green')
    else:
        colors.append('black')

parity_test_fig.add_trace(go.Scatter(
    x=test_set_df['y_test'],
    y=test_set_df['y_test_pred'],
    mode='markers',
    name='Test Set',
    marker=dict(size=6, color=colors)
))

parity_test_fig.add_trace(go.Scatter(
    x=[min(test_set_df['y_test']), max(test_set_df['y_test'])],
    y=[min(test_set_df['y_test']), max(test_set_df['y_test'])],
    mode='lines',
    name='Ideal Prediction',
    line=dict(dash='dash', color='red', width=1.5)
))

# Add custom legend entry
parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='PBE direct'
))

parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='black'),
    name='PBE indirect'
))

parity_test_fig.update_layout(
    template='simple_white',  
    title='Parity plot for HSE Bandgap excluding HSE insulator',
    xaxis_title='Actual Values',
    yaxis_title='Predicted Values',
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    title_x = 0.5,
    title_font_size = 24,
    legend_title_font_color="green",
    xaxis_title_font_family = "arial",
    yaxis_title_font_family = "arial",
    xaxis_title_font_size = 16,
    yaxis_title_font_size = 16,
    showlegend=False,  
    width=1000,  
    height=800,
)

textt = "Black points represent HSE bandgap is indirect<br>Green points represent HSE bandgap is direct"

parity_test_fig.add_annotation(
    text=textt,  
    x= 1,  
    y= 0.05,  
    xref="paper",  
    yref="paper",  
    showarrow=False, 
    font=dict(family="Arial", size=12, color="black"),  
    bgcolor="rgba(255, 255, 255, 0.7)",  
    bordercolor="black",  
    borderwidth=1,  
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

parity_test_fig.show(config=config)

### Feature importances for HSE bandgap prediction excluding HSE insulator

In [163]:
import plotly.graph_objects as go

feature_importances = xgb__reg.feature_importances_
included = X2_HSE.columns.values
indices = np.argsort(feature_importances)[::-1]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=included[indices][:10],
    y=feature_importances[indices][:10],
    marker=dict(color='royalblue'),
    #text=feature_importances[indices][:10],
    #textposition='auto',
))

fig.update_layout(
    xaxis=dict(
        title='Features',
        tickfont=dict(size=14),
        tickangle=45
    ),
    yaxis_title='Importance (%)',
    title='Feature importances for HSE bandgap prediction excluding HSE insulator',
    font=dict(family="Arial", size=14, color="black"),
    title_x = 0.5,
    width=1000,
    height=800,
    showlegend=False,
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)

# For predicting energy above hull distance

### Preparing X,y

In [164]:
excluded_hull = ["A", "B", "functional_group", "training_type", "e_above_hull", "composition", "stable2", "stable", 'HSE_vbm', 'HSE_cbm', 'HSE_direct', "HSE_band_gap", "HSE_insulator", 'PBE_vbm', 'PBE_cbm', 'PBE_direct', 'PBE_band_gap'] #, "total_energy"] #, "stable"] #, "volume", "c"] "HSE_direct", "PBE_direct", 
y = df['e_above_hull'].values
X = df.drop(excluded_hull, axis=1)
print("There are {} possible descriptors:\n\n{}".format(X.shape[1], X.columns.values))

There are 26 possible descriptors:

['A_HOMO' 'A_IE' 'A_IR' 'A_LUMO' 'A_OS' 'A_X' 'A_Z_radii' 'A_e_affin'
 'B_HOMO' 'B_IE' 'B_IR' 'B_LUMO' 'B_OS' 'B_X' 'B_Z_radii' 'B_e_affin' 'mu'
 'tau' 'total_energy' 'efermi' 'density' 'volume' 'PBE_insulator' 'a' 'b'
 'c']


### Model for predicting energy above hull distance

In [165]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming X, y, and df are already defined

# Add 'functional_group' to X
X['functional_group'] = df['functional_group']

# Add 'stable' to X
#X['stable'] = df['stable']

# Specify the test size
test_size = 0.1

# Split the data into train and test sets while maintaining the proportion of 'stable'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=30) # stratify=X['stable'])

# Extract and drop the 'functional_group' and 'stable' columns
train_formula = X_train[['functional_group']]
X_train = X_train.drop(['functional_group'], axis=1)
test_formula = X_test[['functional_group']]
X_test = X_test.drop(['functional_group'], axis=1)

# Create an XGBoost regressor
xgb_reg2 = XGBRegressor(n_estimators=200, random_state=9, learning_rate=0.1)
xgb_reg2.fit(X_train, y_train)

# Get fit statistics
print('Training R2 = ' + str(round(xgb_reg2.score(X_train, y_train), 3)))
print('Training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_train, y_pred=xgb_reg2.predict(X_train))))
print('Test R2 = ' + str(round(xgb_reg2.score(X_test, y_test), 3)))
print('Test RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=xgb_reg2.predict(X_test))))


Training R2 = 1.0
Training RMSE = 0.014
Test R2 = 0.947
Test RMSE = 0.107


### To optimize the model

In [72]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming X, y, and df are already defined

# Add 'functional_group' to X
X['functional_group'] = df['functional_group']

# Add 'stable' to X
#X['stable'] = df['stable']

# Specify the test size
test_size = 0.1

best_r2_score = -float('inf')
best_seed = None

for seed in range(44):
    # Split the data into train and test sets with different random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed) # stratify=X['stable'])

    # Extract and drop the 'functional_group' and 'stable' columns
    train_formula = X_train[['functional_group']]
    X_train = X_train.drop(['functional_group'], axis=1)
    test_formula = X_test[['functional_group']]
    X_test = X_test.drop(['functional_group'], axis=1)

    # Create an XGBoost regressor
    xgb_reg2 = XGBRegressor(n_estimators=200, random_state=9, learning_rate=0.1)
    xgb_reg2.fit(X_train, y_train)

    # Get test R2 score
    test_r2 = xgb_reg2.score(X_test, y_test)
    
    # Update best score and seed if current score is better
    if test_r2 > best_r2_score:
        best_r2_score = test_r2
        best_seed = seed

# Print the best result
print('Best Random Seed:', best_seed)
print('Best Test R2 Score:', best_r2_score)


Best Random Seed: 30
Best Test R2 Score: 0.9471818483556952


### Parity plot for energy above hull - actual vs predicted values

In [168]:
import plotly.graph_objects as go

# Predict on training and test sets
y_train_pred = xgb_reg2.predict(X_train)
y_test_pred = xgb_reg2.predict(X_test)


# Create a parity plot for test set
test_set_df_ = pd.DataFrame({'y_test': y_test, 'y_test_pred': y_test_pred})

# Merge the DataFrame with the main DataFrame 'df' to get 'hse_direct' values
test_set_df_ = pd.merge(test_set_df_, df[['PBE_direct']], left_index=True, right_index=True, how='left')

parity_test_fig = go.Figure()

# Create a color list based on pbe_direct values
colors = "royalblue"

parity_test_fig.add_trace(go.Scatter(
    x=test_set_df_['y_test'],
    y=test_set_df_['y_test_pred'],
    mode='markers',
    name='Test Set',
    marker=dict(size=6, color=colors)
))

parity_test_fig.add_trace(go.Scatter(
    x=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    y=[min(test_set_df_['y_test']), max(test_set_df_['y_test'])],
    mode='lines',
    name='Ideal Prediction',
    line=dict(dash='dash', color='red', width=1.5)
))

# Add custom legend entry
parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='PBE direct'
))

parity_test_fig.add_trace(go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(size=10, color='black'),
    name='PBE indirect'
))

parity_test_fig.update_layout(
    template='simple_white',  # set template to white background
    title='Parity plot for energy above hull - actual vs predicted values',
    xaxis_title='Actual Values',
    yaxis_title='Predicted Values',
    font_family="Courier New",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    title_x = 0.5,
    title_font_size = 24,
    legend_title_font_color="green",
    xaxis_title_font_family = "arial",
    yaxis_title_font_family = "arial",
    yaxis_title_font_size = 16,
    xaxis_title_font_size = 16,
    showlegend=False,  # hide legend
    width=1000,  # Set the width of the plot
    height=800,
)

textt = "Black points represent PBE bandgap is indirect<br>Green points represent PBE bandgap is direct"

#fig.update_xaxes(title_font_family="Arial",tickfont=dict(color='black'))
#fig.update_yaxes(title_font_family="Arial",tickfont=dict(color='black'))

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

parity_test_fig.show(config=config)





### Feature importances for energy above hull prediction model

In [174]:
import plotly.graph_objects as go

feature_importances = xgb_reg2.feature_importances_
included = X.columns.values
indices = np.argsort(feature_importances)[::-1]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=included[indices][:10],
    y=feature_importances[indices][:10],
    marker=dict(color='royalblue'),
))

fig.update_layout(
    xaxis=dict(
        title='Features',
        tickfont=dict(size=12),
        tickangle = 45
    ),
    yaxis_title='Importance (%)',
    title='Feature importances for energy above hull prediction model',
    font=dict(family="Arial", size=12, color="black"),
    width=1000,
    height=800,
    title_x = 0.5,
    showlegend=False,
)

config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 700,
        'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)


## Predicting PBE Bandgap direct or indirect using SVM

In [191]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

to_drop = ["A", "B", "functional_group", "training_type", "PBE_direct", "stable2", "composition", "stable"] #"stable2", "composition"] #, "total_energy"] #, "stable"] #, "volume", "c"]
df_pr = df.dropna()

X_class = df_pr.drop(to_drop, axis=1)
y_class = df_pr['PBE_direct']

X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=16)

# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear')  
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9021739130434783


### Feature importances for model predicting direct/indirect bandgap

In [192]:
import numpy as np
import plotly.graph_objects as go

# Assuming svm_classifier is a trained SVM classifier
# Assuming X_class is your feature matrix

# Get coefficients if SVM is linear
if hasattr(svm_classifier, 'coef_'):
    feature_importances = np.abs(svm_classifier.coef_.flatten())
    included = X_class.columns.values
    indices = np.argsort(feature_importances)[::-1]

    total_importance = np.sum(feature_importances)
    normalized_importances = feature_importances / total_importance
    
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=included[indices][:10],
        y=normalized_importances[indices][:10],
        marker=dict(color='royalblue'),
    ))

    fig.update_layout(
        xaxis=dict(
            title='Features',
            tickfont=dict(size=12),
            tickangle = 45 
        ),
        yaxis_title='Importance',
        title='Feature importances for model predicting PBE direct/indirect bandgap',
        font=dict(family="Arial", size=14, color="black"),
        title_x = 0.5,
        width=1000,
        height=800,
        showlegend=False,
    )

    config = {
        'toImageButtonOptions': {
            'format': 'png',  # one of png, svg, jpeg, webp
            'filename': 'custom_image',
            'height': 500,
            'width': 700,
            'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
        }
    }

    fig.show(config=config)

else:
    print("SVM classifier does not have coefficients to determine feature importances.")


## Predicting HSE Bandgap direct or indirect using SVM

In [188]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

to_drop2 = ["A", "B", "functional_group", "training_type", "HSE_direct", "stable2", "composition"] #, "total_energy"] #, "stable"] #, "volume", "c"]
df_pr = df.dropna()

X_class2 = df_pr.drop(to_drop2, axis=1)
y_class2 = df_pr['HSE_direct']

X_train, X_test, y_train, y_test = train_test_split(X_class2, y_class2, test_size=0.2, random_state=104)

# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear')  
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9239130434782609


### Feature importances for model predicting direct/indirect bandgap

In [190]:
import numpy as np
import plotly.graph_objects as go

if hasattr(svm_classifier, 'coef_'):
    feature_importances = np.abs(svm_classifier.coef_.flatten())
    included = X_class2.columns.values
    indices = np.argsort(feature_importances)[::-1]

    total_importance = np.sum(feature_importances)
    normalized_importances = feature_importances / total_importance
    
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=included[indices][:10],
        y=normalized_importances[indices][:10],
        marker=dict(color='royalblue'),
    ))

    fig.update_layout(
        xaxis=dict(
            title='Features',
            tickfont=dict(size=12),
            tickangle = 45 
        ),
        yaxis_title='Importance',
        title='Feature importances for model predicting HSE direct/indirect bandgap',
        font=dict(family="Arial", size=14, color="black"),
        title_x = 0.5,
        width=1000,
        height=800,
        showlegend=False,
    )

    config = {
        'toImageButtonOptions': {
            'format': 'png',  # one of png, svg, jpeg, webp
            'filename': 'custom_image',
            'height': 500,
            'width': 700,
            'scale': 6  # Multiply title/legend/axis/canvas sizes by this factor
        }
    }

    fig.show(config=config)

else:
    print("SVM classifier does not have coefficients to determine feature importances.")
